In [2]:
%matplotlib inline
import pickle
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
X,y_hr,y_participant,y_activity = pickle.load(open('../data/heart_rate_tabular_data_ppg_dalia.p','rb'))
activity_dict1 = {'No Label':-1,'Sitting':0,'Stairs':1,'Soccer':2,
                'Cycling':3,'Driving':4,'Lunch':-1,'Walking':5,
                'Working':-1}

In [3]:
X = X[:,:,1:].reshape(-1,256,3)
y_activity = np.array([activity_dict1[a] for a in y_activity])
# y_activity = OneHotEncoder().fit_transform(y_activity.reshape(-1,1)).todense()
y_activity.shape

(25144,)

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,LeaveOneGroupOut
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from sklearn.model_selection import train_test_split
# import numpy as np
# %matplotlib notebook
from tensorflow.keras.layers import Conv1D,BatchNormalization,Dropout,Input,MaxPooling1D,Flatten,Dense,Input
from tensorflow.keras.models import Model, Sequential

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)

4 Physical GPUs, 1 Logical GPU


In [4]:
def get_model(input_shape=(256,3),act='tanh',loss="mae",opt='adam',n_classes=1):
    model =  Sequential()
    model.add(Conv1D(100,10,input_shape=input_shape,activation='linear',kernel_initializer='normal',padding='same'))
    model.add(Conv1D(100,10,input_shape=input_shape,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
    model.add(BatchNormalization())
    model.add(Dropout(.2))
    model.add(Conv1D(100,10,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
    model.add(BatchNormalization())
    model.add(Dropout(.2))
    model.add(Conv1D(100,10,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
    model.add(BatchNormalization())
    model.add(Conv1D(200,10,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
    model.add(BatchNormalization())
    model.add(Dropout(.2))
    model.add(Conv1D(50,2,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(4))
    model.add(BatchNormalization())
    model.add(Dropout(.2))
    model.add(Flatten())
    model.add(Dense(n_classes*20,activation='relu',kernel_initializer='normal'))
#     model.add(tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1),name='embedding'))
    model.add(Dense(n_classes,activation=None,kernel_initializer='normal'))
    model.add(Dense(n_classes,activation='softmax',kernel_initializer='normal'))
    model.compile(loss=loss,optimizer=opt)
    return model
model =  get_model(n_classes=6)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 256, 100)          3100      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 256, 100)          100100    
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 128, 100)          0         
_________________________________________________________________
batch_normalization (BatchNo (None, 128, 100)          400       
_________________________________________________________________
dropout (Dropout)            (None, 128, 100)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 128, 100)          100100    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 64, 100)           0

In [ ]:
from keras.models import load_model
n_classes = len(np.unique(y_activity))
model = get_model(n_classes=n_classes,loss='sparse_categorical_crossentropy')
filepath = '../model_files/activity_estimator_8_secs_dalia.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min',save_weights_only=False)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=20)
callbacks_list = [es,checkpoint]
train_x,val_x,train_y,val_y = train_test_split(X,y_activity,test_size=.2,stratify=y_activity)
history = model.fit(train_x,train_y,validation_data=(val_x,val_y), epochs=100, batch_size=100,verbose=1,
      callbacks=callbacks_list,shuffle=True)

In [ ]:
from keras.models import load_model
logo = LeaveOneGroupOut()
y_final = []
y_final_pred = []
y_activity_final = []
y_participant_final = []
n_classes = len(np.unique(y_activity))
for train_index, test_index in logo.split(X, y_hr, y_participant.reshape(-1)):
    train_x, test_x = X[train_index], X[test_index]
    train_y, test_y = y_activity[train_index], y_activity[test_index]
    train_x, val_x, train_y, val_y = train_test_split(train_x,train_y,test_size = 0.1,stratify=train_y)
    print((train_x.shape, train_y.shape), 
          (val_x.shape, val_y.shape),
          (test_x.shape, test_y.shape))    
    model = get_model(n_classes=n_classes,loss='sparse_categorical_crossentropy')
    filepath = '../model_files/temp_activity_estimator.hdf5'
    checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, mode='min',save_weights_only=True)
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=0,patience=20)
    callbacks_list = [es,checkpoint]
    history = model.fit(train_x,train_y,validation_data=(val_x,val_y), epochs=100, batch_size=50,verbose=0,
          callbacks=callbacks_list,shuffle=True)
    model.load_weights(filepath)
    test_y_pred = model.predict(test_x).argmax(axis=1)
    y_final.extend(list(test_y))
    y_final_pred.extend(list(test_y_pred))
    y_participant_final.extend(list(y_participant[test_index]))
    from sklearn.metrics import classification_report
    print(classification_report(test_y,test_y_pred))

In [ ]:
pickle.dump([y_final,y_final_pred,y_participant_final],open('../har_preciction_results.p','wb'))

In [ ]:
y_prediction = model.predict(test_x)
from sklearn.metrics import classification_report
print(classification_report(test_y,y_prediction.argmax(axis=1)))

In [ ]:
y_final,y_final_pred,y_participant_final = pickle.load(open('../har_preciction_results.p','rb'))

In [ ]:
import pandas as pd

In [ ]:
data =pd.DataFrame({'original':y_final,'prediction':y_final_pred,'participant':y_participant_final})

In [ ]:
activity_dict_reverse = {activity_dict1[key]:key for key in activity_dict1.keys()}

In [ ]:
result  = data.groupby(['participant','original'],as_index=False).apply(lambda a:pd.Series({'accuracy':accuracy_score(a['original'],a['prediction'])}))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
result['activity'] = result['original'].apply(lambda a:activity_dict_reverse[a])
result['participant'] = result['participant'].apply(lambda a:'p'+str(a))

In [ ]:
plt.rcParams.update({'font.size':30})
plt.figure(figsize=(20,10))
sns.barplot(x='participant',y='accuracy',data=result)
plt.ylim([0,1])
plt.savefig('./images/partcipant_wise_performance.png')
plt.show()

In [ ]:
plt.rcParams.update({'font.size':30})
plt.figure(figsize=(20,10))
sns.barplot(x='activity',y='accuracy',data=result)
plt.ylim([0,1])
plt.savefig('./images/activity_wise_performance.png')
plt.show()